### [Image Thresholding(图像阈值处理)](https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_thresholding/py_thresholding.html#thresholding)

![](https://docs.opencv.org/3.0-beta/_images/threshold.jpg)

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img=cv2.imread('images/bike.jpg',0)

ret,thresh1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
ret,thresh2 = cv2.threshold(img,127,255,cv2.THRESH_BINARY_INV)
ret,thresh3 = cv2.threshold(img,127,255,cv2.THRESH_TRUNC)
ret,thresh4 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO)
ret,thresh5 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO_INV)

titles = ['Original Image','BINARY','BINARY_INV','TRUNC','TOZERO','TOZERO_INV']
images = [img, thresh1, thresh2, thresh3, thresh4, thresh5]

for i in range(6):
    plt.subplot(2,3,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])

plt.show()


### Adaptive Thresholding(自适应二值化)

In the previous section, we used a global value as threshold value. But it may not be good in all the conditions where image has different lighting conditions in different areas. In that case, we go for adaptive thresholding. In this, the algorithm calculate the threshold for a small regions of the image. So we get different thresholds for different regions of the same image and it gives us better results for images with varying illumination.

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('images/ada_threshold.jpg',0)
# img = cv2.medianBlur(img,5)

ret,th1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,11,2)
th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)

titles = ['Original Image', 'Global Thresholding (v = 127)',
            'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
images = [img, th1, th2, th3]

for i in range(4):
    plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()

### Otsu’s Binarization

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('images/moonlanding.png',0)

# global thresholding
ret1,th1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)

# Otsu's thresholding
ret2,th2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# Otsu's thresholding after Gaussian filtering
blur = cv2.GaussianBlur(img,(11,11),0)
ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# plot all the images and their histograms
images = [img, 0, th1,
          img, 0, th2,
          blur, 0, th3]
titles = ['Original Noisy Image','Histogram','Global Thresholding (v=127)',
          'Original Noisy Image','Histogram',"Otsu's Thresholding",
          'Gaussian filtered Image','Histogram',"Otsu's Thresholding"]

for i in range(3):
    plt.subplot(3,3,i*3+1),plt.imshow(images[i*3],'gray')
    plt.title(titles[i*3]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+2),plt.hist(images[i*3].ravel(),256)
    plt.title(titles[i*3+1]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+3),plt.imshow(images[i*3+2],'gray')
    plt.title(titles[i*3+2]), plt.xticks([]), plt.yticks([])
plt.show()

# [Smoothing Images](https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_filtering/py_filtering.html#filtering)

## cv2.filter2D( )是通用滤波函数

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('images/opencv_logo.png')

redis=31

#均值滤波（Averaging）
kernel = np.ones((redis,redis),np.float32)/(redis*redis)
dst = cv2.filter2D(img,-1,kernel)


def show_bgr(img):
    plt.subplot()
    plt.imshow(img[:,:,::-1])
    plt.show()
    
show_bgr(dst)

In [ ]:
#Averaging（均值滤波）

averaging=cv2.blur(img,(redis,redis))
show_bgr(np.hstack([img,averaging]))

In [ ]:
#Gaussian Blurring（高斯滤波）

gaussian_blurring=cv2.GaussianBlur(img,(redis,redis),0)
show_bgr(np.hstack([img,gaussian_blurring]))

In [ ]:
#Median Blurring(中值滤波)
#用邻域中亮度的中值取代图像当前的点
#中值滤波方法是，对待处理的当前像素，选择一个模板，该模板为其邻近的若干个像素组成，对模板的像素由小到大进行排序，再用模板的中值来替代原像素的值的方法。

moonlanding=cv2.imread('images/moonlanding.png')
median=cv2.medianBlur(moonlanding,redis)
show_bgr(np.hstack([moonlanding,median]))
cv2.imwrite('images/moonlanding_median_blurring.png',median)

###  Bilateral Filtering（双边滤波）
>cv2.bilateralFilter() is highly effective in noise removal while keeping edges sharp. But the operation is slower compared to other filters.

对于高斯滤波，仅用空间距离的权值系数核与图像卷积后，确定中心点的灰度值。即认为离中心点越近的点，其权重系数越大。

双边滤波中加入了对灰度信息的权重，即在邻域内，灰度值越接近中心点灰度值的点的权重更大，灰度值相差大的点权重越小。此权重大小，则由值域高斯函数确定。

两者权重系数相乘，得到最终的卷积模板。由于双边滤波需要每个中心点邻域的灰度信息来确定其系数，所以其速度与比一般的滤波慢很多，而且计算量增长速度为核大小的平方。



In [ ]:
opencv_logo_noise=cv2.imread('images/opencv_logo_noise.jpg')

gaussian_blur=cv2.GaussianBlur(opencv_logo_noise,(21,21),0)
bilateral_blur=cv2.bilateralFilter(opencv_logo_noise,21,200,200)
show_bgr(np.hstack([opencv_logo_noise,gaussian_blur,bilateral_blur]))